In [230]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import locale
from locale import atof

In [231]:
# Load in csv
filepath = os.path.join("CSV Files\Zillow CSV\Price.csv")
price_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\Zillow CSV\Rent.csv")
rent_df = pd.read_csv(filepath)

In [232]:
# Convert Region Name into Seperate City and State Columns
rent_df['City'],rent_df['State'] = rent_df["RegionName"].str.split(',', 2).str[0:2].str
rent_df['State'].replace(' TX','TX', inplace=True)
rent_df.head()

# Bottom lines were pushed town to focus on Rent above
#price_df['City'],price_df['State'] = price_df["RegionName"].str.split(',', 2).str[0:2].str
#price_df['State'].replace(' TX','TX', inplace=True)
#price_df.head()

,RegionID,RegionName,SizeRank,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,...,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,City,State
0,102001,United States,0,1244.0,1247.0,1248.0,1247.0,1246.0,1245.0,1244.0,...,1394,1397,1399,1401,1405,1410,1414,1417,United States,NaN
1,394913,"New York, NY",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2414,2407,2402,2404,2409,2411,2411,2417,New York,NY
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,2275.0,2269.0,2268.0,2268.0,2268.0,2271.0,2272.0,...,2771,2778,2790,2804,2816,2824,2830,2834,Los Angeles-Long Beach-Anaheim,CA
3,394463,"Chicago, IL",3,1571.0,1565.0,1564.0,1560.0,1554.0,1547.0,1539.0,...,1670,1676,1685,1691,1695,1695,1697,1700,Chicago,IL
4,394514,"Dallas-Fort Worth, TX",4,1295.0,1299.0,1303.0,1307.0,1308.0,1309.0,1309.0,...,1600,1600,1600,1602,1607,1614,1619,1623,Dallas-Fort Worth,TX


In [233]:
# Extract only the Texas Data
rent_tx_df = rent_df.loc[rent_df["State"] == "TX", :]
rent_tx_df = rent_tx_df.set_index('City')
rent_tx_df.head()

# Bottom lines were pushed town to focus on Rent above
#price_tx_df = price_df.loc[price_df["State"] == "TX", :]
#price_tx_df = price_tx_df.set_index('City')
#price_tx_df.head()

,RegionID,RegionName,SizeRank,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,...,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,State
City,,,,,,,,,,,,,,,,,,,,,
Dallas-Fort Worth,394514,"Dallas-Fort Worth, TX",4,1295.0,1299.0,1303.0,1307.0,1308.0,1309.0,1309.0,...,1597,1600,1600,1600,1602,1607,1614,1619,1623,TX
Houston,394692,"Houston, TX",6,1365.0,1365.0,1366.0,1368.0,1371.0,1373.0,1371.0,...,1564,1562,1561,1558,1555,1554,1554,1555,1557,TX
San Antonio,395055,"San Antonio, TX",25,1157.0,1153.0,1153.0,1151.0,1147.0,1140.0,1136.0,...,1335,1337,1339,1340,1340,1339,1338,1340,1341,TX
Austin,394355,"Austin, TX",33,1306.0,1318.0,1329.0,1341.0,1350.0,1356.0,1359.0,...,1712,1710,1712,1717,1723,1728,1727,1723,1719,TX
El Paso,394561,"El Paso, TX",65,1087.0,1088.0,1087.0,1077.0,1063.0,1052.0,1051.0,...,993,992,991,992,993,995,995,995,996,TX


In [234]:
# Remove Undesired Columns
rent_tx_df = rent_tx_df.drop(rent_tx_df.columns[[0,1,2,-1]], axis=1)
rent_tx_df.head()


# Bottom lines were pushed town to focus on Rent above
#price_tx_df = price_tx_df.drop(price_tx_df.columns[[0,1,-1]], axis=1)
#price_tx_df.head()

,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,...,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10
City,,,,,,,,,,,,,,,,,,,,,
Dallas-Fort Worth,1295.0,1299.0,1303.0,1307.0,1308.0,1309.0,1309.0,1310.0,1311.0,1310.0,...,1594,1597,1600,1600,1600,1602,1607,1614,1619,1623
Houston,1365.0,1365.0,1366.0,1368.0,1371.0,1373.0,1371.0,1366.0,1361.0,1356.0,...,1570,1564,1562,1561,1558,1555,1554,1554,1555,1557
San Antonio,1157.0,1153.0,1153.0,1151.0,1147.0,1140.0,1136.0,1132.0,1130.0,1130.0,...,1334,1335,1337,1339,1340,1340,1339,1338,1340,1341
Austin,1306.0,1318.0,1329.0,1341.0,1350.0,1356.0,1359.0,1362.0,1362.0,1361.0,...,1718,1712,1710,1712,1717,1723,1728,1727,1723,1719
El Paso,1087.0,1088.0,1087.0,1077.0,1063.0,1052.0,1051.0,1054.0,1051.0,1048.0,...,994,993,992,991,992,993,995,995,995,996


In [235]:
# Transpose Data Frame
rent = rent_tx_df.transpose()
rent.head()


# Bottom lines were pushed town to focus on Rent above
#price = price_tx_df.transpose()

City,Dallas-Fort Worth,Houston,San Antonio,Austin,El Paso,McAllen,Corpus Christi,Brownsville,Killeen,Beaumont,...,Brenham,Kingsville,Fredericksburg,Pampa,Dumas,Snyder,Sweetwater,Andrews,Pecos,Vernon
2010-11,1295.0,1365.0,1157.0,1306.0,1087.0,1010.0,1158.0,940.0,1099.0,1136.0,...,1238.0,1011.0,NaN,952.0,948.0,905.0,960.0,1089.0,964.0,1006.0
2010-12,1299.0,1365.0,1153.0,1318.0,1088.0,1003.0,1161.0,936.0,1102.0,1131.0,...,1240.0,1010.0,NaN,960.0,944.0,911.0,966.0,1091.0,967.0,1006.0
2011-01,1303.0,1366.0,1153.0,1329.0,1087.0,1007.0,1156.0,937.0,1083.0,1126.0,...,1243.0,1010.0,NaN,962.0,929.0,905.0,982.0,1091.0,969.0,1011.0
2011-02,1307.0,1368.0,1151.0,1341.0,1077.0,1004.0,1152.0,933.0,1058.0,1115.0,...,1243.0,1007.0,NaN,973.0,926.0,906.0,994.0,1099.0,977.0,1016.0
2011-03,1308.0,1371.0,1147.0,1350.0,1063.0,996.0,1143.0,936.0,1028.0,1108.0,...,1246.0,1004.0,NaN,984.0,926.0,909.0,996.0,1104.0,975.0,1026.0


In [236]:
#Move the Index
rent.reset_index(level=0, inplace=True)
rent = rent.rename(columns = {'index':'Date'})
rent = rent.reset_index(drop=True)
rent.head()

City,Date,Dallas-Fort Worth,Houston,San Antonio,Austin,El Paso,McAllen,Corpus Christi,Brownsville,Killeen,...,Brenham,Kingsville,Fredericksburg,Pampa,Dumas,Snyder,Sweetwater,Andrews,Pecos,Vernon
0,2010-11,1295.0,1365.0,1157.0,1306.0,1087.0,1010.0,1158.0,940.0,1099.0,...,1238.0,1011.0,NaN,952.0,948.0,905.0,960.0,1089.0,964.0,1006.0
1,2010-12,1299.0,1365.0,1153.0,1318.0,1088.0,1003.0,1161.0,936.0,1102.0,...,1240.0,1010.0,NaN,960.0,944.0,911.0,966.0,1091.0,967.0,1006.0
2,2011-01,1303.0,1366.0,1153.0,1329.0,1087.0,1007.0,1156.0,937.0,1083.0,...,1243.0,1010.0,NaN,962.0,929.0,905.0,982.0,1091.0,969.0,1011.0
3,2011-02,1307.0,1368.0,1151.0,1341.0,1077.0,1004.0,1152.0,933.0,1058.0,...,1243.0,1007.0,NaN,973.0,926.0,906.0,994.0,1099.0,977.0,1016.0
4,2011-03,1308.0,1371.0,1147.0,1350.0,1063.0,996.0,1143.0,936.0,1028.0,...,1246.0,1004.0,NaN,984.0,926.0,909.0,996.0,1104.0,975.0,1026.0


In [237]:
# Convert the Date into a Month and Year Column
rent['Year'],rent['Month'] = rent["Date"].str.split('-', 2).str[0:2].str
rent = rent.drop(rent.columns[[0]], axis=1)
rent.head()

City,Dallas-Fort Worth,Houston,San Antonio,Austin,El Paso,McAllen,Corpus Christi,Brownsville,Killeen,Beaumont,...,Fredericksburg,Pampa,Dumas,Snyder,Sweetwater,Andrews,Pecos,Vernon,Year,Month
0,1295.0,1365.0,1157.0,1306.0,1087.0,1010.0,1158.0,940.0,1099.0,1136.0,...,NaN,952.0,948.0,905.0,960.0,1089.0,964.0,1006.0,2010,11
1,1299.0,1365.0,1153.0,1318.0,1088.0,1003.0,1161.0,936.0,1102.0,1131.0,...,NaN,960.0,944.0,911.0,966.0,1091.0,967.0,1006.0,2010,12
2,1303.0,1366.0,1153.0,1329.0,1087.0,1007.0,1156.0,937.0,1083.0,1126.0,...,NaN,962.0,929.0,905.0,982.0,1091.0,969.0,1011.0,2011,01
3,1307.0,1368.0,1151.0,1341.0,1077.0,1004.0,1152.0,933.0,1058.0,1115.0,...,NaN,973.0,926.0,906.0,994.0,1099.0,977.0,1016.0,2011,02
4,1308.0,1371.0,1147.0,1350.0,1063.0,996.0,1143.0,936.0,1028.0,1108.0,...,NaN,984.0,926.0,909.0,996.0,1104.0,975.0,1026.0,2011,03


In [259]:
# DELETE IF IT DOES NOT WORK (melt)
# IT WORKS!!!!
rent_melt = pd.melt(rent, id_vars=["Year", "Month"])
rent_melt = rent_melt.rename(columns={'value': 'Rent'})
rent_melt = rent_melt.replace('Dallas-Fort Worth', 'Dallas')
#rent_melt = rent_melt.sort_values(by=['Year','Month','City'])
rent_melt.head()

,Year,Month,City,Rent
0,2010,11,Dallas,1295.0
1,2010,12,Dallas,1299.0
2,2011,01,Dallas,1303.0
3,2011,02,Dallas,1307.0
4,2011,03,Dallas,1308.0


In [239]:
# Convert Region Name into Seperate City and State Columns

price_df['City'],price_df['State'] = price_df["RegionName"].str.split(',', 2).str[0:2].str
price_df['State'].replace(' TX','TX', inplace=True)
price_df.head()

,RegionName,SizeRank,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,...,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,City,State
0,United States,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,249900.0,254900.0,259900.0,259990.0,259900.0,259000.0,259900.0,259900.0,United States,NaN
1,"New York, NY",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,439999.0,449000.0,458894.0,459900.0,459000.0,449869.0,450000.0,459000.0,New York,NY
2,"Los Angeles, CA",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,650000.0,672245.0,679000.0,670000.0,675000.0,675000.0,675000.0,670000.0,Los Angeles,CA
3,"Chicago, IL",3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,259500.0,265000.0,269000.0,269900.0,269000.0,262000.0,260000.0,259900.0,Chicago,IL
4,"Dallas, TX",4,164900.0,164900.0,165000.0,166000.0,165000.0,165332.5,164900.0,165000.0,...,312500.0,319000.0,320000.0,320000.0,319328.0,315000.0,319000.0,318815.0,Dallas,TX


In [240]:
# Extract only the Texas Data

price_tx_df = price_df.loc[price_df["State"] == "TX", :]
price_tx_df = price_tx_df.set_index('City')
price_tx_df.head()

,RegionName,SizeRank,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,...,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,State
City,,,,,,,,,,,,,,,,,,,,,
Dallas,"Dallas, TX",4,164900.0,164900.0,165000.0,166000.0,165000.0,165332.5,164900.0,165000.0,...,300482.5,312500.0,319000.0,320000.0,320000.0,319328.0,315000.0,319000.0,318815.0,TX
Houston,"Houston, TX",6,159000.0,159900.0,160000.0,160000.0,159900.0,159900.0,159000.0,159900.0,...,285000.0,289900.0,293990.0,294900.0,293990.0,289900.0,289900.0,289999.0,289900.0,TX
San Antonio,"San Antonio, TX",26,159900.0,162500.0,162500.0,163881.0,164900.0,167085.5,165000.0,164436.5,...,259900.0,269000.0,271495.0,270489.5,275000.0,275000.0,274900.0,274900.0,266000.0,TX
Austin,"Austin, TX",35,204402.0,203450.0,200595.0,204900.0,205000.0,206333.0,200000.0,204900.0,...,330000.0,336777.5,344990.0,345000.0,345000.0,342489.0,339500.0,335900.0,335000.0,TX
El Paso,"El Paso, TX",68,139950.0,140000.0,139800.0,140000.0,142950.0,144694.0,144900.0,143000.0,...,158500.0,159900.0,159900.0,160000.0,161950.0,161500.0,161450.0,161950.0,163000.0,TX


In [241]:
# Remove Undesired Columns

price_tx_df = price_tx_df.drop(price_tx_df.columns[[0,1,-1]], axis=1)
price_tx_df.head()

,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,...,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10
City,,,,,,,,,,,,,,,,,,,,,
Dallas,164900.0,164900.0,165000.0,166000.0,165000.0,165332.5,164900.0,165000.0,178900.0,171900.0,...,299900.0,300482.5,312500.0,319000.0,320000.0,320000.0,319328.0,315000.0,319000.0,318815.0
Houston,159000.0,159900.0,160000.0,160000.0,159900.0,159900.0,159000.0,159900.0,170990.0,169900.0,...,280990.0,285000.0,289900.0,293990.0,294900.0,293990.0,289900.0,289900.0,289999.0,289900.0
San Antonio,159900.0,162500.0,162500.0,163881.0,164900.0,167085.5,165000.0,164436.5,169150.0,169000.0,...,254900.0,259900.0,269000.0,271495.0,270489.5,275000.0,275000.0,274900.0,274900.0,266000.0
Austin,204402.0,203450.0,200595.0,204900.0,205000.0,206333.0,200000.0,204900.0,214900.0,210900.0,...,325000.0,330000.0,336777.5,344990.0,345000.0,345000.0,342489.0,339500.0,335900.0,335000.0
El Paso,139950.0,140000.0,139800.0,140000.0,142950.0,144694.0,144900.0,143000.0,142950.0,141900.0,...,158000.0,158500.0,159900.0,159900.0,160000.0,161950.0,161500.0,161450.0,161950.0,163000.0


In [245]:
# Transpose Data Frame
price = price_tx_df.transpose()
price.head()

City,Dallas,Houston,San Antonio,Austin,El Paso,Edinburg,Corpus Christi,Brownsville,Killeen,Beaumont,...,Uvalde,Fredericksburg,Levelland,Pampa,Borger,Port Lavaca,Snyder,Sweetwater,Andrews,Lamesa
2010-01,164900.0,159000.0,159900.0,204402.0,139950.0,120000.0,164900.0,172500.0,137900.0,139900.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02,164900.0,159900.0,162500.0,203450.0,140000.0,125000.0,164500.0,174700.0,138800.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03,165000.0,160000.0,162500.0,200595.0,139800.0,124999.5,164900.0,172250.0,139400.0,133632.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04,166000.0,160000.0,163881.0,204900.0,140000.0,127000.0,164100.0,169900.0,139900.0,134900.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05,165000.0,159900.0,164900.0,205000.0,142950.0,119900.0,164900.0,174500.0,138000.0,134900.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [246]:
#Move the Index
price.reset_index(level=0, inplace=True)
price = price.rename(columns = {'index':'Date'})
price = price.reset_index(drop=True)
price.head()

City,Date,Dallas,Houston,San Antonio,Austin,El Paso,Edinburg,Corpus Christi,Brownsville,Killeen,...,Uvalde,Fredericksburg,Levelland,Pampa,Borger,Port Lavaca,Snyder,Sweetwater,Andrews,Lamesa
0,2010-01,164900.0,159000.0,159900.0,204402.0,139950.0,120000.0,164900.0,172500.0,137900.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-02,164900.0,159900.0,162500.0,203450.0,140000.0,125000.0,164500.0,174700.0,138800.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-03,165000.0,160000.0,162500.0,200595.0,139800.0,124999.5,164900.0,172250.0,139400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-04,166000.0,160000.0,163881.0,204900.0,140000.0,127000.0,164100.0,169900.0,139900.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-05,165000.0,159900.0,164900.0,205000.0,142950.0,119900.0,164900.0,174500.0,138000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [247]:
# Convert the Date into a Month and Year Column
price['Year'],price['Month'] = price["Date"].str.split('-', 2).str[0:2].str
price = price.drop(price.columns[[0]], axis=1)
price.head()

City,Dallas,Houston,San Antonio,Austin,El Paso,Edinburg,Corpus Christi,Brownsville,Killeen,Beaumont,...,Levelland,Pampa,Borger,Port Lavaca,Snyder,Sweetwater,Andrews,Lamesa,Year,Month
0,164900.0,159000.0,159900.0,204402.0,139950.0,120000.0,164900.0,172500.0,137900.0,139900.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,01
1,164900.0,159900.0,162500.0,203450.0,140000.0,125000.0,164500.0,174700.0,138800.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,02
2,165000.0,160000.0,162500.0,200595.0,139800.0,124999.5,164900.0,172250.0,139400.0,133632.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,03
3,166000.0,160000.0,163881.0,204900.0,140000.0,127000.0,164100.0,169900.0,139900.0,134900.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,04
4,165000.0,159900.0,164900.0,205000.0,142950.0,119900.0,164900.0,174500.0,138000.0,134900.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,05


In [256]:
# DELETE IF IT DOES NOT WORK (melt)
# IT WORKS!!!!
price_melt = pd.melt(price, id_vars=["Year", "Month"])
price_melt = price_melt.rename(columns={'value': 'House Price'})
#price_melt = price_melt.sort_values(by=['Year','Month','City'])
price_melt.head()

,Year,Month,City,House Price
0,2010,01,Dallas,164900.0
1,2010,02,Dallas,164900.0
2,2010,03,Dallas,165000.0
3,2010,04,Dallas,166000.0
4,2010,05,Dallas,165000.0


In [261]:
merge_melts = pd.merge(rent_melt,price_melt, how='outer', on=['Year','Month','City'])
merge_melts.head()

,Year,Month,City,Rent,House Price
0,2010,11,Dallas,1295.0,165000.0
1,2010,12,Dallas,1299.0,159900.0
2,2011,01,Dallas,1303.0,159000.0
3,2011,02,Dallas,1307.0,159000.0
4,2011,03,Dallas,1308.0,162990.0
